# Préparation des données

Montage google drive et décompression des données de bdappv.zip

In [15]:
import sys
from google.colab import drive
drive.mount('/content/drive/', force_remount=True)
#sys.path.insert(0,'/content/drive/My Drive/statapps-main/src/')
!unzip /content/drive/MyDrive/bdappv/bdappv.zip > /dev/null

Mounted at /content/drive/
replace __MACOSX/._bdappv? [y]es, [n]o, [A]ll, [N]one, [r]ename: A
[A]ll


Importation du fichier dataloader pour pouvoir utiliser ses modules

In [ ]:
!cp /content/drive/MyDrive/statapps-main/src/dataloader.py /content

In [16]:
import dataloader as dt
import torchvision.models as models

In [19]:
label_attribution = dt.LabelAttribution(path_image_google='/content/bdappv/google/img', 
                                   path_mask_google='/content/bdappv/google/mask',
                                   path_metadata='/content/bdappv/metadata.csv',
                                   colonne_identifiant='identifiant',
                                   path_export_train_test='/content/drive/MyDrive',
                                   path_image_ign='/content/bdappv/ign/img',
                                   path_mask_ign='/content/bdappv/ign/mask',
                                   use_img_google=True,
                                   use_img_ign=False
                                    )

In [20]:
label_attribution.run()

In [ ]:
models.resnet18(pretrained=False, progress=True)